In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [7]:
pd.read_sql(
    '''
    SELECT * FROM Customers
    ''',
    con,
)

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021,Mexico
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,5023,Mexico
3,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK
4,5,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,Luleå,S-958 22,Sweden
...,...,...,...,...,...,...,...
86,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland
87,88,Wellington Importadora,Paula Parente,"Rua do Mercado, 12",Resende,08737-363,Brazil
88,89,White Clover Markets,Karl Jablonski,305 - 14th Ave. S. Suite 3B,Seattle,98128,USA
89,90,Wilman Kala,Matti Karttunen,Keskuskatu 45,Helsinki,21240,Finland


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address
    FROM Customers
    WHERE Country = 'Germany'
       or Country = 'France'
       or City = 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [9]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country, COUNT(CustomerName) AS Count
    FROM Customers
    GROUP BY Country
    ORDER BY Count DESC
    LIMIT 3
    ''',
    con,
)

,Country,Count
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [10]:
tasks+=1
pd.read_sql(
    '''
    SELECT ShipperName, OrderDate
    FROM Orders
    JOIN Shippers ON Orders.ShipperID = Shippers.ShipperID
    ORDER BY OrderDate LIMIT 1
    OFFSET 9
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,02.10.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [11]:
tasks+=1
pd.read_sql(
    '''
    SELECT ProductName, Price
    FROM OrderDetails
    JOIN Products ON OrderDetails.ProductID = Products.ProductID
    WHERE OrderID =
    (
        SELECT OrderID
        FROM OrderDetails
            JOIN Products ON OrderDetails.ProductID = Products.ProductID
        GROUP BY OrderID
        ORDER BY SUM(Quantity * Price) DESC
    LIMIT 1
    )
    ''',
    con,
)

,ProductName,Price
0,Sir Rodney's Marmalade,81.0
1,Côte de Blaye,263.5
2,Camembert Pierrot,34.0
3,Mozzarella di Giovanni,34.8


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [12]:
tasks+=1
pd.read_sql(
    '''
    SELECT OrderID, ProductName, Quantity
    FROM OrderDetails
    JOIN Products ON OrderDetails.ProductID = Products.ProductID
    WHERE OrderDetails.ProductID =
    (
        SELECT ProductID
        FROM OrderDetails
        GROUP BY ProductID
        ORDER BY SUM(Quantity) DESC
    LIMIT 1
    )
    ''',
    con,
)

,OrderID,ProductName,Quantity
0,10253,Gorgonzola Telino,20
1,10272,Gorgonzola Telino,40
2,10273,Gorgonzola Telino,15
3,10325,Gorgonzola Telino,4
4,10335,Gorgonzola Telino,25
5,10342,Gorgonzola Telino,56
6,10356,Gorgonzola Telino,30
7,10359,Gorgonzola Telino,70
8,10363,Gorgonzola Telino,20
9,10374,Gorgonzola Telino,30


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT SupplierName, Country, Phone
    FROM OrderDetails
        JOIN Products ON OrderDetails.ProductID = Products.ProductID
        JOIN Suppliers ON Suppliers.SupplierID = Products.SupplierID
    GROUP BY SupplierName
    ORDER BY COUNT (OrderID) DESC
    LIMIT 5
    ''',
    con,
)

,SupplierName,Country,Phone
0,Plutzer Lebensmittelgroßmärkte AG,Germany,(069) 992755
1,"Pavlova, Ltd.",Australia,(03) 444-2343
2,"Specialty Biscuits, Ltd.",UK,(161) 555-4448
3,Norske Meierier,Norway,(0)2-953010
4,Formaggi Fortini s.r.l.,Italy,(0544) 60323


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [14]:
tasks+=1
pd.read_sql(
    '''
    SELECT CategoryName, Country, SUM(Price * Quantity) AS Sum
    FROM OrderDetails
        JOIN Orders ON OrderDetails.OrderID = Orders.OrderID
        JOIN Customers ON Orders.CustomerID = Customers.CustomerID
        JOIN Products ON OrderDetails.ProductID = Products.ProductID
        JOIN Categories ON Products.CategoryID = Categories.CategoryID
    WHERE Customers.Country = 'Brazil'
    GROUP BY Categories.CategoryID
    ORDER BY SUM (Price * Quantity) DESC
    LIMIT 1
    ''',
    con,
)

,CategoryName,Country,Sum
0,Beverages,Brazil,13690.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [15]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country, MAX(Price * Quantity) - MIN(Price * Quantity) AS Diff
    FROM OrderDetails
         JOIN Orders ON OrderDetails.OrderID = Orders.OrderID
         JOIN Customers ON Orders.CustomerID = Customers.CustomerID
         JOIN Products ON OrderDetails.ProductID = Products.ProductID
    WHERE Customers.Country = 'USA'
    ''',
    con,
)

,Country,Diff
0,USA,5232.5


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [16]:
tasks+=1
pd.read_sql(
    '''
    SELECT Employees.FirstName || ' ' || Employees.LastName AS EmployeeName, COUNT(OrderID) AS OrdersCount
    FROM Orders
         JOIN Employees ON Employees.EmployeeID = Orders.EmployeeID
    GROUP BY Orders.EmployeeID
    ORDER BY Employees.BirthDate DESC
    LIMIT 3
    ''',
    con,
)

,EmployeeName,OrdersCount
0,Janet Leverling,31
1,Robert King,14
2,Margaret Peacock,40


10. Сколько банок крабового мяса всего было заказано.

In [17]:
tasks+=1
pd.read_sql(
    '''
    SELECT SUM(Quantity) * (SELECT CAST(SUBSTR(Unit, 6, 1) AS INTEGER)
    FROM Products
    WHERE ProductName = 'Boston Crab Meat') AS TotalBostonCrabMeatTinsSold
    FROM OrderDetails
         JOIN Products ON OrderDetails.ProductID = Products.ProductID
    WHERE ProductName = 'Boston Crab Meat'
    ''',
    con,
)

,TotalBostonCrabMeatTinsSold
0,1024


In [18]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
